## Forward Propagation in RNNs

In [1]:
import numpy as np
import tensorflow as tf
import math
import random

In [2]:
input_dim = 7
hidden_dim = 7
output_dim = 7

In [3]:
U = np.random.uniform(-np.sqrt(1.0 / input_dim), np.sqrt(1.0 / input_dim), (hidden_dim, input_dim))
W = np.random.uniform(-np.sqrt(1.0 / hidden_dim), np.sqrt(1.0 / hidden_dim), (hidden_dim, hidden_dim))
V = np.random.uniform(-np.sqrt(1.0 / hidden_dim), np.sqrt(1.0 / hidden_dim), (input_dim, hidden_dim))

In [4]:
x =[0,1]

In [5]:
num_time_steps = len(x)


In [6]:
hidden_state = np.zeros((num_time_steps + 1, hidden_dim))

In [7]:
hidden_state[-1] = np.zeros(hidden_dim)

In [8]:
YHat = np.zeros((num_time_steps, output_dim))

In [9]:
def softmax(x):
  return np.exp(x) / np.exp(x).sum(axis=0)

In [10]:
for t in np.arange(num_time_steps):
  #h_t = tanh(UX + Wh_{t-1})
  hidden_state[t] = np.tanh(U[:, x[t]] + W.dot(hidden_state[t - 1]))
  # yhat_t = softmax(vh)
  YHat[t] = softmax(V.dot(hidden_state[t]))

## Generating Somg lyrics using RNN

In [11]:
import warnings
warnings.filterwarnings('ignore')

import random
import numpy as np
import tensorflow as tf
import pandas as pd

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)

import warnings
warnings.filterwarnings('ignore')

In [12]:
df = pd.read_csv('/content/data/songdata.csv')

In [13]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [14]:
df.shape[0]

57650

In [15]:
len(df['artist'].unique())

643

In [16]:
df['artist'].value_counts()[:10]

Donna Summer        191
Gordon Lightfoot    189
Bob Dylan           188
George Strait       188
Loretta Lynn        187
Cher                187
Alabama             187
Reba Mcentire       187
Chaka Khan          186
Dean Martin         186
Name: artist, dtype: int64

In [17]:
df['artist'].value_counts().values.mean()

89.65785381026438

In [18]:
data = ', '.join(df['text'])

In [19]:
data[:369]

"Look at her face, it's a wonderful face  \nAnd it means something special to me  \nLook at the way that she smiles when she sees me  \nHow lucky can one fellow be?  \n  \nShe's just my kind of girl, she makes me feel fine  \nWho could ever believe that she could be mine?  \nShe's just my kind of girl, without her I'm blue  \nAnd if she ever leaves me what could I do, what co"

In [20]:
chars = sorted(list(set(data)))

In [21]:
vocab_size = len(chars)

In [22]:
vocab_size

76

In [23]:
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

In [24]:
print(char_to_ix['s'])

68


In [25]:
print(ix_to_char[68])

s


In [26]:
vocabSize = 7
char_index = 4
print(np.eye(vocabSize)[char_index])


[0. 0. 0. 0. 1. 0. 0.]


In [27]:
def one_hot_encoder(index):
  return np.eye (vocab_size)[index]

In [28]:
hidden_size = 100

In [29]:
seq_length = 25
learning_rate = 1e-1

In [30]:
seed_value = 42
tf.compat.v1.set_random_seed(seed_value)
random.seed(seed_value)

In [31]:
tf.compat.v1.disable_eager_execution()
inputs = tf.compat.v1.placeholder(shape=[None, vocab_size], dtype=tf.float32, name="inputs")
targets = tf.compat.v1.placeholder(shape=[None, vocab_size], dtype=tf.float32, name="targets")


In [32]:
init_state = tf.compat.v1.placeholder(shape=[1, hidden_size], dtype=tf.float32, name="state")

In [33]:
initializer = tf.random_normal_initializer(stddev=0.1)

In [34]:
with tf.compat.v1.variable_scope("RNN") as scope:
    h_t = init_state
    y_hat = []

    for t, x_t in enumerate(tf.split(inputs, seq_length, axis=0)):
        if t > 0:
            scope.reuse_variables()

        #input to hidden layer weights
        U = tf.compat.v1.get_variable("U", [vocab_size, hidden_size], initializer=initializer)
        #hidden to hidden layer weights
        W = tf.compat.v1.get_variable("W", [hidden_size, hidden_size], initializer=initializer)
        #output to hidden layer weights
        V = tf.compat.v1.get_variable("V", [hidden_size, vocab_size], initializer=initializer)
        #bias for hidden layer
        bh = tf.compat.v1.get_variable("bh", [hidden_size], initializer=initializer)
        #bias for output layer
        by = tf.compat.v1.get_variable("by", [vocab_size], initializer=initializer)
        h_t = tf.tanh(tf.matmul(x_t, U) + tf.matmul(h_t, W) + bh)
        y_hat_t = tf.matmul(h_t, V) + by
        y_hat.append(y_hat_t)

In [35]:
output_softmax = tf.nn.softmax(y_hat[-1])
outputs = tf.concat(y_hat, axis=0)

In [36]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=targets, logits=outputs))

In [37]:
hprev = h_t

In [38]:
minimizer = tf.compat.v1.train.AdamOptimizer()

In [39]:
gradients = minimizer.compute_gradients(loss)

In [40]:
threshold = tf.constant(5.0, name="grad_clipping")

In [41]:
clipped_gradients = []
for grad, var in gradients:
    clipped_grad = tf.clip_by_value(grad, -threshold, threshold)
    clipped_gradients.append((clipped_grad, var))

In [42]:
updated_gradients = minimizer.apply_gradients(clipped_gradients)

In [43]:
sess = tf.compat.v1.Session()
init = tf.compat.v1.global_variables_initializer()
sess.run(init)

In [44]:
pointer = 0
iteration = 0

In [45]:
while iteration < 50001:

    if pointer + seq_length+1 >= len(data) or iteration == 0:
        hprev_val = np.zeros([1, hidden_size])
        pointer = 0

    #select input sentence
    input_sentence = data[pointer:pointer + seq_length]

    #select output sentence
    output_sentence = data[pointer + 1:pointer + seq_length + 1]

    #get the indices of input and output sentence
    input_indices = [char_to_ix[ch] for ch in input_sentence]
    target_indices = [char_to_ix[ch] for ch in output_sentence]

    #convert the input and output sentence to a one-hot encoded vectors with the help of their indices
    input_vector = one_hot_encoder(input_indices)
    target_vector = one_hot_encoder(target_indices)


    #train the network and get the final hidden state
    hprev_val, loss_val, _ = sess.run([hprev, loss, updated_gradients],
                                      feed_dict={inputs: input_vector,targets: target_vector,init_state: hprev_val})


    #make predictions on every 500th iteration
    if iteration % 500 == 0:

        #length of characters we want to predict
        sample_length = 500

        #randomly select index
        random_index = random.randint(0, len(data) - seq_length)

        #sample the input sentence with the randomly selected index
        sample_input_sent = data[random_index:random_index + seq_length]

        #get the indices of the sampled input sentence
        sample_input_indices = [char_to_ix[ch] for ch in sample_input_sent]

        #store the final hidden state in sample_prev_state_val
        sample_prev_state_val = np.copy(hprev_val)

        #for storing the indices of predicted characters
        predicted_indices = []


        for t in range(sample_length):

            #convert the sampled input sentence into one-hot encoded vector using their indices
            sample_input_vector = one_hot_encoder(sample_input_indices)

            #compute the probability of all the words in the vocabulary to be the next character
            probs_dist, sample_prev_state_val = sess.run([output_softmax, hprev],
                                                      feed_dict={inputs: sample_input_vector,init_state: sample_prev_state_val})

            #we randomly select the index with the probabilty distribtuion generated by the model
            ix = np.random.choice(range(vocab_size), p=probs_dist.ravel())

            sample_input_indices = sample_input_indices[1:] + [ix]


            #store the predicted index in predicted_indices list
            predicted_indices.append(ix)

        #convert the predicted indices to their character
        predicted_chars = [ix_to_char[ix] for ix in predicted_indices]

        #combine the predcited characters
        text = ''.join(predicted_chars)

        #predict the predict text on every 50000th iteration
        if iteration % 5000 == 0:
            print ('\n')
            print (' After %d iterations' %(iteration))
            print('\n %s \n' % (text,))
            print('-'*115)


    #increment the pointer and iteration
    pointer += seq_length
    iteration += 1



 After 0 iterations

 r42]74[DMv6[Un4nCJ::mi9NEKHTv4?4VdarpZPZM'rJMKGKrKM]c6!g37MSQSKBzKz4!wguwn,JpG.froVn1x-zzLu?2
HG0(Db'0AGt7W g
yCLt-Xuw 03pB,ZY[ySP3D6I9Mc7ZqX6WJbS!0KCdHTD)i:tSqd9oycMga5hWlDowk)-NyZmGq(O!R[SAS(3adAsgFf,agTIFV(K,9fiZZDNu Uu?,?niQHOj'GyWxgPXf0,kPZWphQeEW7Mn)p!06sK,4 QzE'N9Bg)igiphgM:HKElVcGGRQE?ueD)LheMq9"O6m?G,gwfTInplov8AVG2BXT5AmBt1K8]00rWx5FU
(xZ1BHv[E0] "9S,HmS-QalllS]'vIIUMVr31zkH36DT,9Wjl!pF8(vx!0Jy0D!FvnB?ug6?KPr95r)VtU]:b9bJML?GE9do"WvtNXFe9Wjkr4Te96mz4fmQjJhPMHUW]YRxXscGkPP-oK)6[b8f9daMR 

-------------------------------------------------------------------------------------------------------------------


 After 5000 iterations

 n of te te wour tay, on the tuned sill habe  
Thee s aasp tnithous 
And  
Who  lane ay  
The aop I fon the mistsine be -
I'm sthe w on drodsthey, shite  
I  che taldk por  
I  heake on lele githe Coushes me it it de me tor came  
I Fakndtsire  
I m Iving t  
I suge to s a plet eveeve ot lone, ot don't ou han  ie wy a ghet you da